## Lab 4.3. 

#### Setup your imports

In [3]:
import pandas as pd 
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import f_classif, SelectKBest
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import matplotlib.pyplot as plt

In [4]:
print "Goal: Figure out what newsgroup each block of text belongs to and create list of 20 different categories"

Goal: Figure out what newsgroup each block of text belongs to and create list of 20 different categories


In [5]:
# Steps To Take:

# 1) Gather and clean data

# 2) Turn text into numbers

# 3) Plot numbers

# 4) 

#### 1. Pull the training set from the newsgroup data
The data has 20 different categories. Try to shrink down to smaller number of groups according to the definition here:
http://scikit-learn.org/stable/datasets/twenty_newsgroups.html

In [2]:
newsgroups_train = fetch_20newsgroups(subset='train', remove =['headers','footers','quotes'])

x = newsgroups_train.data # Text data (emails, etc)
y = newsgroups_train.target #Newsgroup

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.
7


In [109]:
print "List of 20 different categories:"
pprint(list(newsgroups_train.target_names))

List of 20 different categories:
['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .33, random_state = 42)

In [9]:
# Reduce to 6 targets

mappy = {
    0: [0],
    1: [1,2,3,4,5],
    2: [6],
    3: [7,8,9,10],
    4: [11,12,13,14],
    5: [15],
    6: [16,17,18,19]
}
def getkey(num):
    for x, y in mappy.items():
        if num in y:
            return x
y_train = [getkey(num) for num in y_train]
y_test = [getkey(num) for num in y_test]

labels = ['alt','comp','misc','rec','sci','soc','talk']

print y_train[0:10]


[3, 3, 6, 6, 4, 6, 1, 4, 4, 4]


#### 2. Create the vectorizer 

In [10]:
# Create vectorizer of words
v = CountVectorizer(stop_words = 'english', ngram_range = (1,2), max_features=1000)

# Fit Vectorizer
vec = v.fit(x)
x_train = vec.transform(x_train).todense()
x_test = vec.transform(x_test).todense()

# Visualize the vectorizer (Rows are newsgroups, columns are earch word)
df_x_train = pd.DataFrame(x_train, columns = list(v.vocabulary_))

df_x_train.head()

,learn,runs,night,person,similar,room,teams,armenia,like,des,...,market,command,probably,sounds,talk,shot,week,d9,general,cover
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


What are top 50 most powerful terms in deciding news groups? (hint: treat it as a classification problem)?

In [11]:
# Using Logistic Regression and get coeficients

# Scale the data
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

# Create model
lr = LogisticRegression(multi_class='multinomial', solver='sag')
lr.fit(x_train, y_train)

# Make predictions
y_predictions = lr.predict(x_test)


/Users/Madballa55/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)
/Users/Madballa55/anaconda/lib/python2.7/site-packages/sklearn/linear_model/sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [21]:
# Return accuracy
print "Score: ", lr.score(x_test, y_test)
print "Confusion Matrix"
conf_matrix = confusion_matrix(y_test, y_predictions)
print conf_matrix

print classification_report(y_test, y_predictions)



Score:  0.664702731655
Confusion Matrix
[[ 24  25   0  17  16   9  57]
 [  0 847   5  31  61   1   3]
 [  0  37  99  29  22   0   1]
 [  1 157   6 547  51   2  25]
 [  3 169   2  94 475   0  52]
 [  4  18   1  15  13  84  67]
 [  5  82   2  73  71  25 406]]
             precision    recall  f1-score   support

          0       0.65      0.16      0.26       148
          1       0.63      0.89      0.74       948
          2       0.86      0.53      0.65       188
          3       0.68      0.69      0.69       789
          4       0.67      0.60      0.63       795
          5       0.69      0.42      0.52       202
          6       0.66      0.61      0.64       664

avg / total       0.67      0.66      0.65      3734



In [24]:
# Create DF with coefecients and words
df_coef = pd.DataFrame(lr.coef_, columns = list(v.vocabulary_))

# Sort by top words
abs(df_coef.ix[6]).sort_values(ascending = False)[0:10]

df_coef.head()

,learn,runs,night,person,similar,room,teams,armenia,like,des,...,market,command,probably,sounds,talk,shot,week,d9,general,cover
0,-0.000570,0.007567,0.002878,0.003168,0.005718,0.006668,0.004404,0.004680,0.004030,-0.001678,...,0.030704,0.063545,0.004834,0.004141,-0.019603,-0.000860,-0.013873,0.026469,-0.000928,0.011300
1,-0.008796,-0.042217,0.002859,0.001681,0.007635,-0.020511,0.009618,0.008233,0.005382,-0.019449,...,0.018270,-0.045802,0.036949,0.035984,-0.061782,-0.075862,-0.047421,0.017046,-0.032135,-0.034048
2,0.069203,0.021646,0.001001,0.001164,0.004099,0.004834,0.002242,0.004393,0.004638,0.034097,...,-0.010186,-0.008227,0.006129,0.009851,-0.002797,0.005069,-0.005627,-0.031964,0.005325,0.006774
3,-0.004532,0.023355,0.004628,0.007347,0.000915,-0.005811,-0.004519,-0.003917,-0.002980,0.008199,...,-0.047211,-0.026451,-0.013795,-0.015364,0.180301,0.019292,0.017791,-0.020591,0.035043,0.022053
4,-0.035638,-0.026250,-0.009564,-0.011868,-0.019396,0.015293,-0.011865,-0.012997,-0.010871,-0.016693,...,-0.046479,-0.036416,-0.029279,-0.025776,-0.049712,0.061001,0.034011,-0.007011,-0.041070,-0.036123


In [23]:
# Find top 50 using K-Best 
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(f_classif, k=50)
selected_data = selector.fit_transform(x, y)
kbest_columns = x.columns[selector.get_support()]
df_Xbest = pd.DataFrame(selected_data, columns=kbest_columns)

df_Xbest.head()

/Users/Madballa55/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: Found input variables with inconsistent numbers of samples: [1, 11314]

#### 3. Create the Truncated Singular Value Decomposition. 

In [154]:
# Train, Test, Split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .33, random_state = 42)

# Create vectorizer of words
v = CountVectorizer(stop_words = 'english', ngram_range = (1,2), token_pattern='[a-zA-z]{3,50}', max_features=1000)

# Fit Vectorizer
x_vec = v.fit(x)
x_train = vec.transform(x_train)
x_test = vec.transform(x_test)

In [149]:
tsvd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)
tsvd.fit(x_train)

print tsvd.explained_variance_ratio_

print "Accuracy: ", sum(tsvd.explained_variance_ratio_)

[ 0.5703765   0.06492919  0.03048025  0.02542723  0.02096249  0.01442626
  0.01354996  0.01176197  0.0116698   0.01049065  0.01033613  0.00886223
  0.00765617  0.00737933  0.00639738  0.00537683  0.00518967  0.00481976
  0.00407646  0.00397765  0.00386957  0.00362885  0.00337894  0.00311016
  0.00297092  0.00266808  0.00250592  0.00232504  0.00222912  0.00220091
  0.00207813  0.0020416   0.00200061  0.00196643  0.00195494  0.00187375
  0.00184024  0.00180891  0.00173396  0.00165543  0.00155811  0.00149014
  0.00143859  0.00140879  0.00132653  0.00122297  0.0011927   0.00112166
  0.00109169  0.00106718]
Accuracy:  0.898905766593


#### 4. Setup your k-means clustering

In [155]:
x_train = tsvd.transform(x_train)

from sklearn import cluster
k = 7
kmeans = cluster.KMeans(n_clusters=k)
kmeans.fit(x_train)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=7, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [156]:
labels_predictions = kmeans.labels_
labels_predictions

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

#### 5. Fit the vectorizer and SVD

What are top 50 most useful terms based on article itself? Are those terms similar to the top 50 from step 2? 

Plot the cummulative variace from the terms. x-axis: number of components; y-axis: cummulative variance. 
Based on the plot, decide how many principle components you need. 

#### 7. Fit the kmeans (Question: in this case, do you recommend running K-means without dimension reduction?)

Print out your centroids. Look at the value for each centroid. Does each centroid represent a news group as expected? 

#### 8. Check the performance of our kmeans

#### Classification Report

#### Confusion Matrix. Hint: create a map to translate the label between k-means clustering and the original target (newsgroups_train.target). 

#### Note: Repeat the lab with:
- varying values of "k" 
- trying a different way to pick starting centroids ('k-means++' is the default method for centroids). For example, pick one point from each newsgroup. 